# Data Preparation for Weather, Stocks, and Crawler Data for Flight Analysis
This notebook combines weather data, stock market data, and flight crawler data to generate the dataset for our flight analysis. It includes cleaning and transforming airport and city names, date correction, and merging the datasets based on airport and date fields. Additionally, we ensure consistent data formats and manage missing values to provide accurate flight insights.

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

In [1]:
# Load the combined datasets for crawler data, weather and stocks
crawler_daten = pd.read_csv('data/cralwer_data_merged.csv')
weather = pd.read_csv('data/combined_weather.csv')
stocks = pd.read_csv('data/combined_stocks.csv')

In [3]:
# Drop unnecessary weather columns
weather.drop(columns=['tsun', 'snow'], inplace=True)

# Mapping for city names in the weather data to airport codes
city_mapping = {
    'Frankfurt': 'FRA',
    'Berlin': 'BER',
    'Hamburg': 'HAM',
    'München': 'MUC',
    'Muenchen': 'MUC',  # Alternative spelling
    'München': 'MUC',
    'Munich': 'MUC',
    'London': 'LHR',
    'Palma': 'PMI',
    'Palma de Mallorca': 'PMI',
    'Istanbul': 'SAW',
    'Dubai': 'DXB',
    'New York': 'JFK',
    'New_York': 'JFK',
    'Shanghai': 'PVG'
}

# Convert full city names to their respective airport codes
weather['City'] = weather['City'].replace(city_mapping)
# Convert city names to airport codes in crawler data
crawler_daten['departure_airport'] = crawler_daten['departure_airport'].replace(city_mapping)
crawler_daten['destination_airport'] = crawler_daten['destination_airport'].replace(city_mapping)

# Check if all entries were correctly converted
print("Unique City values after mapping:", weather['City'].unique())

Unique City values after mapping: ['FRA' 'BER' 'PMI' 'DXB' 'PVG' 'MUC' 'JFK' 'HAM' 'LHR' 'SAW']


In [4]:
crawler_daten.columns

Index(['airline_name', 'crawling_date', 'departure_airport',
       'destination_airport', 'date', 'travel_duration', 'departure_time',
       'arrival_time', 'transit', 'transit_duration', 'price'],
      dtype='object')

In [5]:
# Split the weather DataFrame into destination and departure weather data
weather_destination = weather[weather['City'] != 'FRA']
weather_departure = weather[weather['City'] == 'FRA']

In [6]:
# Ensure the date format is consistent across datasets
crawler_daten['date'] = pd.to_datetime(crawler_daten['date']).dt.date
weather_departure['Date'] = pd.to_datetime(weather_departure['Date']).dt.date

# Check if the values in the 'departure_airport' and 'City' columns as well as 'date' and 'Date' match
print("Unique departure_airport in crawler_daten:", crawler_daten['departure_airport'].unique())
print("Unique City in weather_departure:", weather_departure['City'].unique())

# Compare date values
print("Unique dates in crawler_daten:", crawler_daten['date'].unique())
print("Unique dates in weather_departure:", weather_departure['Date'].unique())

Unique departure_airport in crawler_daten: ['FRA']
Unique City in weather_departure: ['FRA']
Unique dates in crawler_daten: [datetime.date(2024, 6, 8) datetime.date(2024, 7, 8)
 datetime.date(2024, 8, 8) datetime.date(2024, 9, 8)
 datetime.date(2024, 10, 8) datetime.date(2024, 11, 8)
 datetime.date(2024, 12, 8) datetime.date(2024, 8, 13)
 datetime.date(2024, 8, 14) datetime.date(2024, 8, 15)
 datetime.date(2024, 8, 16) datetime.date(2024, 8, 17)
 datetime.date(2024, 8, 18) datetime.date(2024, 8, 19)
 datetime.date(2024, 8, 20) datetime.date(2024, 8, 21)
 datetime.date(2024, 8, 22) datetime.date(2024, 8, 23)
 datetime.date(2024, 8, 24) datetime.date(2024, 8, 25)
 datetime.date(2024, 8, 26) datetime.date(2024, 8, 27)
 datetime.date(2024, 8, 28) datetime.date(2024, 8, 29)
 datetime.date(2024, 8, 30) datetime.date(2024, 8, 31)
 datetime.date(2024, 1, 9)]
Unique dates in weather_departure: [datetime.date(2024, 8, 1) datetime.date(2024, 8, 2)
 datetime.date(2024, 8, 3) datetime.date(2024, 8,

/Users/sami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '13-08-2024' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/sami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '14-08-2024' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/sami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '15-08-2024' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
/Users/sami/opt/anaconda3/lib/python3.9/site-packages/pandas/core/tools/datetimes.py:1047: UserWarning: Parsing '16-08-2024' in

In [11]:
# Function to correct swapped date values
def correct_date(date_value):
    # Check if the date is a swapped date
    if date_value.month == 8 and date_value.day != 8:
        return date_value
    elif date_value.day == 8 and date_value.month != 8:
        # Fix the error by swapping day and month
        return datetime(date_value.year, 8, date_value.month).date()
    else:
        return date_value

# Apply the correction function to the 'date' column in crawler_daten
crawler_daten['date'] = crawler_daten['date'].apply(correct_date)

# Verify the correction
print("Eindeutige Datumswerte in crawler_daten nach Korrektur:", crawler_daten['date'].unique())

Eindeutige Datumswerte in crawler_daten nach Korrektur: [datetime.date(2024, 8, 6) datetime.date(2024, 8, 7)
 datetime.date(2024, 8, 8) datetime.date(2024, 8, 9)
 datetime.date(2024, 8, 10) datetime.date(2024, 8, 11)
 datetime.date(2024, 8, 12) datetime.date(2024, 8, 13)
 datetime.date(2024, 8, 14) datetime.date(2024, 8, 15)
 datetime.date(2024, 8, 16) datetime.date(2024, 8, 17)
 datetime.date(2024, 8, 18) datetime.date(2024, 8, 19)
 datetime.date(2024, 8, 20) datetime.date(2024, 8, 21)
 datetime.date(2024, 8, 22) datetime.date(2024, 8, 23)
 datetime.date(2024, 8, 24) datetime.date(2024, 8, 25)
 datetime.date(2024, 8, 26) datetime.date(2024, 8, 27)
 datetime.date(2024, 8, 28) datetime.date(2024, 8, 29)
 datetime.date(2024, 8, 30) datetime.date(2024, 8, 31)
 datetime.date(2024, 1, 9)]


In [12]:
# Ensure the date format is consistent across datasets
crawler_daten['date'] = pd.to_datetime(crawler_daten['date']).dt.date
weather_departure['Date'] = pd.to_datetime(weather_departure['Date']).dt.date

# Merge the corrected data
merged_departure = pd.merge(
    crawler_daten, 
    weather_departure, 
    how='left', 
    left_on=['departure_airport', 'date'], 
    right_on=['City', 'Date']
)

# Rename weather columns for the departure airport
departure_weather_columns = {col: f"{col}_departure" for col in weather_departure.columns if col not in ['City', 'Date']}
merged_departure.rename(columns=departure_weather_columns, inplace=True)

# Drop unnecessary columns after the merge (City and Date)
merged_departure.drop(columns=['City', 'Date'], inplace=True)

# Display result
print("Merged Data Sample:")
print(merged_departure.head(20))  # Print the first 20 rows for verification

Merged Data Sample:
        airline_name crawling_date departure_airport destination_airport  \
0   AustrianAirlines    2024-08-05               FRA                 BER   
1       QatarAirways    2024-08-05               FRA                 LHR   
2       QatarAirways    2024-08-05               FRA                 HAM   
3       QatarAirways    2024-08-05               FRA                 BER   
4                KLM    2024-08-05               FRA                 BER   
5          Lufthansa    2024-08-05               FRA                 PVG   
6          Lufthansa    2024-08-05               FRA                 JFK   
7          Lufthansa    2024-08-05               FRA                 DXB   
8          Lufthansa    2024-08-05               FRA                 SAW   
9          Lufthansa    2024-08-05               FRA                 PMI   
10      QatarAirways    2024-08-05               FRA                 SAW   
11         Lufthansa    2024-08-05               FRA                

/var/folders/x8/7crcl1z12n7bm3r7zkc6gn540000gn/T/ipykernel_28914/781761017.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_departure['Date'] = pd.to_datetime(weather_departure['Date']).dt.date


In [13]:
# Ensure the date format is consistent in weather_destination
weather_destination['Date'] = pd.to_datetime(weather_destination['Date']).dt.date

# Merge the data for the destination airport
merged_final = pd.merge(
    merged_departure, 
    weather_destination, 
    how='left', 
    left_on=['destination_airport', 'date'], 
    right_on=['City', 'Date']
)

# Rename the columns for destination weather data
destination_weather_columns = {col: f"{col}_destination" for col in weather_destination.columns if col not in ['City', 'Date']}
merged_final.rename(columns=destination_weather_columns, inplace=True)

# Drop unnecessary columns after the merge (City and Date)
merged_final.drop(columns=['City', 'Date'], inplace=True)

# Save the final result in 'crawled_data_weather_df'
crawled_data_weather_df = merged_final

# Display the final merged data
print("Final Merged Data Sample with Destination Weather:")
print(crawled_data_weather_df.head(20))  # Print the first 20 rows for verification

Final Merged Data Sample with Destination Weather:
        airline_name crawling_date departure_airport destination_airport  \
0   AustrianAirlines    2024-08-05               FRA                 BER   
1       QatarAirways    2024-08-05               FRA                 LHR   
2       QatarAirways    2024-08-05               FRA                 HAM   
3       QatarAirways    2024-08-05               FRA                 BER   
4                KLM    2024-08-05               FRA                 BER   
5          Lufthansa    2024-08-05               FRA                 PVG   
6          Lufthansa    2024-08-05               FRA                 JFK   
7          Lufthansa    2024-08-05               FRA                 DXB   
8          Lufthansa    2024-08-05               FRA                 SAW   
9          Lufthansa    2024-08-05               FRA                 PMI   
10      QatarAirways    2024-08-05               FRA                 SAW   
11         Lufthansa    2024-08-05   

/var/folders/x8/7crcl1z12n7bm3r7zkc6gn540000gn/T/ipykernel_28914/1257553672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  weather_destination['Date'] = pd.to_datetime(weather_destination['Date']).dt.date


In [15]:
# Verify the unique date values in crawled_data_weather_df
print("Eindeutige Datumswerte in crawled_data_weather_df:")
print(sorted(crawled_data_weather_df['date'].unique()))

# Check for missing date values (e.g., certain August days)
missing_dates = set(pd.date_range(start='2024-08-01', end='2024-08-31').date).difference(set(crawled_data_weather_df['date']))
print("\nFehlende Datumswerte in crawled_data_weather_df für August 2024:")
print(sorted(missing_dates))

Eindeutige Datumswerte in crawled_data_weather_df:
[datetime.date(2024, 1, 9), datetime.date(2024, 8, 6), datetime.date(2024, 8, 7), datetime.date(2024, 8, 8), datetime.date(2024, 8, 9), datetime.date(2024, 8, 10), datetime.date(2024, 8, 11), datetime.date(2024, 8, 12), datetime.date(2024, 8, 13), datetime.date(2024, 8, 14), datetime.date(2024, 8, 15), datetime.date(2024, 8, 16), datetime.date(2024, 8, 17), datetime.date(2024, 8, 18), datetime.date(2024, 8, 19), datetime.date(2024, 8, 20), datetime.date(2024, 8, 21), datetime.date(2024, 8, 22), datetime.date(2024, 8, 23), datetime.date(2024, 8, 24), datetime.date(2024, 8, 25), datetime.date(2024, 8, 26), datetime.date(2024, 8, 27), datetime.date(2024, 8, 28), datetime.date(2024, 8, 29), datetime.date(2024, 8, 30), datetime.date(2024, 8, 31)]

Fehlende Datumswerte in crawled_data_weather_df für August 2024:
[datetime.date(2024, 8, 1), datetime.date(2024, 8, 2), datetime.date(2024, 8, 3), datetime.date(2024, 8, 4), datetime.date(2024, 8,

In [16]:
stocks['Date'] = pd.to_datetime(stocks['Date']).dt.date
stocks

,Date,Open,High,Low,Close,Adj Close,Volume,Airline
0,2024-08-01,8.170,8.170,7.836,7.926,7.926,1473798.0,KLM
1,2024-08-01,5.708,5.798,5.588,5.654,5.654,7842242.0,Lufthansa
2,2024-08-01,293.000,298.250,293.000,297.500,297.500,21064407.0,Turkish
3,2024-08-02,7.924,7.974,7.788,7.944,7.944,1538347.0,KLM
4,2024-08-02,5.620,5.662,5.492,5.562,5.562,7692759.0,Lufthansa
...,...,...,...,...,...,...,...,...
88,2024-08-30,5.920,5.954,5.858,5.888,5.888,5924554.0,Lufthansa
89,2024-08-31,7.968,8.244,7.968,8.222,8.222,1916188.0,KLM
90,2024-08-31,5.920,5.954,5.858,5.888,5.888,5924554.0,Lufthansa
91,2024-09-01,7.968,8.244,7.968,8.222,8.222,1916188.0,KLM


In [ ]:
# Map AustrianAirlines to Lufthansa since no stock data is available
crawled_data_weather_df['airline_name'] = crawled_data_weather_df['airline_name'].replace({'AustrianAirlines': 'Lufthansa'})

# Filter the data to include only valid overlapping dates
valid_dates = set(stocks['Date']).intersection(set(crawled_data_weather_df['date']))
filtered_crawled_data_weather_df = crawled_data_weather_df[crawled_data_weather_df['date'].isin(valid_dates)]

# Merge filtered crawler and weather data with stock data based on airline and date
final_df = pd.merge(filtered_crawled_data_weather_df, stocks, how='left', left_on=['airline_name', 'date'], right_on=['Airline', 'Date'])

# Remove unnecessary columns after the merge
final_df.drop(columns=['Airline', 'Date'], inplace=True)

In [ ]:
# Save the final dataset
final_df.to_csv('data/analysis_dataset.csv')